In [17]:
import pandas as pd
from io import StringIO
import requests
from tqdm import tqdm
import ast

df=pd.read_csv(r"datasets/01a_Sentence_df.csv", sep="|", lineterminator='\n')
df["taggedSentences"]  = [ast.literal_eval(text) for text in df["taggedSentences"]]
display(df.head())

# Sentence numbers uncluding sentences with only punctuation
# A     7141
# B    20087
# C    19318

df = df[df["Text"] != '.']
df = df[df["Text"] != '!']
df = df[df["Text"] != ':']
df = df[df["Text"] != '?' ]
df = df[df["Text"] != '*']
df = df[df["Text"] != ","]
df = df[df["Text"] != '-']
df = df[df["Text"] != '.,']
df = df[df["Text"] != '. ,']
df = df[df['taggedSentences'].apply(lambda x: len(x)>1)]

df['Level_ABC'] = df.apply (lambda row: (row["newLevel"]),axis=1)
df = df[df.Level_ABC != "L"]
level = df.groupby('Level_ABC')
print(level.size())
df = level.apply(lambda x: x.sample(level.size().min()).reset_index(drop=True))
print(len(df[df["newLevel"]=="A"] ))

,Text,newLevel,taggedSentences,parsedSentences
0,Zu meiner Familie gehören vier Personen.,A,"[[Zu, APPR, Dat], [meiner, PRO, Poss, Attr, -,...",(ROOT (S (PP (APPR Zu) (PPOSAT meiner) (NN Fam...
1,Die Mutter bin ich und dann gehört natürlich m...,A,"[[Die, ART, Def, Nom, Sg, Fem], [Mutter, N, Re...",(ROOT (CS (S (NP (ART Die) (NN Mutter)) (VAFIN...
2,"Wir haben zwei Kinder, einen Sohn, der sechs J...",A,"[[Wir, PRO, Pers, Subst, 1, Nom, Pl, *], [habe...",(ROOT (S (PPER Wir) (VAFIN haben) (CNP (NP (CA...
3,Wir wohnen in einem kleinen Haus mit einem Gar...,A,"[[Wir, PRO, Pers, Subst, 1, Nom, Pl, *], [wohn...",(ROOT (S (PPER Wir) (VVFIN wohnen) (PP (APPR i...
4,Dort können die Kinder ein bisschen spielen.,A,"[[Dort, ADV], [können, VFIN, Mod, 3, Pl, Pres,...",(ROOT (S (ADV Dort) (VMFIN können) (NP (ART di...


Level_ABC
A     6901
B    19660
C    19025
dtype: int64
6901


In [18]:
from sklearn.model_selection import train_test_split

def save_df(path,df):
    df.to_csv(path,sep="|",index=False)
    

x_train ,x_test = train_test_split(df,test_size=0.3)

for df_inProcessing, df_name in tqdm([(x_train, "train"), (x_test, "test")]):
    for N in tqdm(range(1,10)): # number of sentences per Text
        ###Text with len N sentence###
        rows = []

        ###split training and test data###


        complexityLevels = ['A', 'B', 'C']
        
        no_of_text = min([int(len(df_inProcessing[(df["newLevel"] == complexityLevel)])/N) for complexityLevel in complexityLevels])
        print(no_of_text)
        for complexityLevel in complexityLevels:
            
            for n in tqdm(range(no_of_text)): #number of text per level
                df_sample = df_inProcessing[(df["newLevel"] == complexityLevel)].sample(N)
                rows.append([" ".join(df_sample['Text'].tolist()), complexityLevel, df_sample['taggedSentences'].tolist(), df_sample['parsedSentences'].tolist()])
        random_df = pd.DataFrame(rows, columns=['Text', 'newLevel', 'RFTagger', 'parsedText'])

        display(random_df.head())

        filename = str("datasets/RandomText/10_RandomText_" + df_name + "_" + str(N) + "_df.csv")
        save_df(filename, random_df)

  0%|          | 0/9 [00:00<?, ?it/s]/home/soeren/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


  0%|          | 0/4822 [00:00<?, ?it/s]/home/soeren/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


  0%|          | 17/4822 [00:00<00:29, 162.06it/s]

4822




  1%|          | 36/4822 [00:00<00:27, 171.51it/s]

  1%|          | 55/4822 [00:00<00:27, 174.60it/s]

  2%|▏         | 73/4822 [00:00<00:27, 175.22it/s]

  2%|▏         | 92/4822 [00:00<00:26, 176.43it/s]

  2%|▏         | 111/4822 [00:00<00:26, 177.11it/s]

  3%|▎         | 130/4822 [00:00<00:26, 177.97it/s]

  3%|▎         | 149/4822 [00:00<00:26, 178.79it/s]

  3%|▎         | 168/4822 [00:00<00:25, 179.73it/s]

  4%|▍         | 188/4822 [00:01<00:25, 180.78it/s]

  4%|▍         | 207/4822 [00:01<00:25, 180.02it/s]

  5%|▍         | 225/4822 [00:01<00:26, 176.19it/s]

  5%|▌         | 242/4822 [00:01<00:26, 175.58it/s]

  5%|▌         | 259/4822 [00:01<00:26, 174.64it/s]

  6%|▌         | 277/4822 [00:01<00:26, 174.42it/s]

  6%|▌         | 294/4822 [00:01<00:26, 173.68it/s]

  6%|▋         | 311/4822 [00:01<00:26, 172.41it/s]

  7%|▋         | 330/4822 [00:01<00:25, 173.02it/s]

  7%|▋         | 348/4822 [00:02<00:25, 173.31it/s]

  8%|▊         | 366/4822 [00:02<00:25, 172.92it

 52%|█████▏    | 2529/4822 [00:14<00:13, 171.87it/s]

 53%|█████▎    | 2547/4822 [00:14<00:13, 171.84it/s]

 53%|█████▎    | 2565/4822 [00:14<00:13, 171.83it/s]

 54%|█████▎    | 2583/4822 [00:15<00:13, 171.83it/s]

 54%|█████▍    | 2601/4822 [00:15<00:12, 171.63it/s]

 54%|█████▍    | 2618/4822 [00:15<00:12, 171.61it/s]

 55%|█████▍    | 2635/4822 [00:15<00:12, 171.56it/s]

 55%|█████▌    | 2654/4822 [00:15<00:12, 171.68it/s]

 55%|█████▌    | 2674/4822 [00:15<00:12, 171.81it/s]

 56%|█████▌    | 2693/4822 [00:15<00:12, 171.91it/s]

 56%|█████▌    | 2712/4822 [00:15<00:12, 172.02it/s]

 57%|█████▋    | 2731/4822 [00:15<00:12, 172.11it/s]

 57%|█████▋    | 2751/4822 [00:15<00:12, 172.24it/s]

 57%|█████▋    | 2770/4822 [00:16<00:11, 172.35it/s]

 58%|█████▊    | 2789/4822 [00:16<00:11, 172.43it/s]

 58%|█████▊    | 2808/4822 [00:16<00:11, 172.53it/s]

 59%|█████▊    | 2827/4822 [00:16<00:11, 172.62it/s]

 59%|█████▉    | 2846/4822 [00:16<00:11, 172.69it/s]

 59%|█████▉    | 2865/4822 [

  1%|          | 47/4822 [00:00<00:31, 151.08it/s]

  1%|▏         | 65/4822 [00:00<00:30, 156.59it/s]

  2%|▏         | 81/4822 [00:00<00:30, 156.72it/s]

  2%|▏         | 95/4822 [00:00<00:32, 147.55it/s]

  2%|▏         | 110/4822 [00:00<00:32, 146.90it/s]

  3%|▎         | 126/4822 [00:00<00:31, 147.65it/s]

  3%|▎         | 140/4822 [00:00<00:32, 145.40it/s]

  3%|▎         | 154/4822 [00:01<00:32, 143.88it/s]

  4%|▎         | 169/4822 [00:01<00:32, 143.74it/s]

  4%|▍         | 184/4822 [00:01<00:32, 143.81it/s]

  4%|▍         | 199/4822 [00:01<00:32, 143.79it/s]

  5%|▍         | 217/4822 [00:01<00:31, 145.76it/s]

  5%|▍         | 232/4822 [00:01<00:31, 145.71it/s]

  5%|▌         | 247/4822 [00:01<00:31, 145.24it/s]

  5%|▌         | 262/4822 [00:01<00:31, 144.67it/s]

  6%|▌         | 277/4822 [00:01<00:31, 144.07it/s]

  6%|▌         | 292/4822 [00:02<00:31, 144.09it/s]

  6%|▋         | 307/4822 [00:02<00:31, 143.94it/s]

  7%|▋         | 322/4822 [00:02<00:31, 143.85it/s

...


 89%|████████▉ | 8/9 [01:22<00:10, 10.28s/it]

  0%|          | 0/228 [00:00<?, ?it/s]

 10%|█         | 23/228 [00:00<00:00, 220.36it/s]

228




 21%|██        | 47/228 [00:00<00:00, 226.48it/s]

 31%|███       | 71/228 [00:00<00:00, 229.04it/s]

 42%|████▏     | 96/228 [00:00<00:00, 233.69it/s]

 53%|█████▎    | 121/228 [00:00<00:00, 236.45it/s]

 64%|██████▍   | 146/228 [00:00<00:00, 238.28it/s]

 75%|███████▌  | 171/228 [00:00<00:00, 239.68it/s]

 86%|████████▌ | 196/228 [00:00<00:00, 239.81it/s]

 97%|█████████▋| 221/228 [00:00<00:00, 240.39it/s]

100%|██████████| 228/228 [00:00<00:00, 239.93it/s]

  0%|          | 0/228 [00:00<?, ?it/s]

 11%|█         | 25/228 [00:00<00:00, 243.95it/s]

 22%|██▏       | 50/228 [00:00<00:00, 241.77it/s]

 31%|███       | 70/228 [00:00<00:00, 227.44it/s]

 41%|████      | 94/228 [00:00<00:00, 228.98it/s]

 52%|█████▏    | 118/228 [00:00<00:00, 230.95it/s]

 63%|██████▎   | 143/228 [00:00<00:00, 233.08it/s]

 73%|███████▎  | 166/228 [00:00<00:00, 232.42it/s]

 83%|████████▎ | 189/228 [00:00<00:00, 231.45it/s]

 93%|█████████▎| 213/228 [00:00<00:00, 232.11it/s]

100%|██████████| 228/228 [00

,Text,newLevel,RFTagger,parsedText
0,Ich hatte gern Besuch komm aber Die Wiener Uni...,A,"[[[Ich, PRO, Pers, Subst, 1, Nom, Sg, *], [hat...",[(ROOT (NUR (S (PPER Ich) (VAFIN hatte) (AVP (...
1,"Er bekommt morgens seine Hundekuchen, aber mit...",A,"[[[Er, PRO, Pers, Subst, 3, Nom, Sg, Masc], [b...",[(ROOT (CS (S (PPER Er) (VVFIN bekommt) (ADV m...
2,Felle Danke. Tut mir leid. Das Wetter war nich...,A,"[[[Felle, N, Reg, Acc, Pl, Neut], [Danke, PART...","[(ROOT (NP (ADJA Felle) (NN Danke) ($. .))), (..."
3,"Bis Morgen meine Liebe Maria Der Skilehrer, He...",A,"[[[Bis, CONJ, SubFin, -], [Morgen, ADV], [mein...",[(ROOT (NUR (S (PP (APPR Bis) (ADV Morgen)) (N...
4,"Leider Können ich nicht fahren, weil mein Mann...",A,"[[[Leider, ADV], [Können, VFIN, Mod, 3, Sg, Pr...",[(ROOT (S (VP (NP (ADV Leider) (NN Können)) (N...



100%|██████████| 2/2 [06:00<00:00, 180.38s/it]


In [ ]:
int(len(df_inProcessing[(df["newLevel"] == complexityLevel)])/5))